In [ ]:
!npm install -g localtunnel
%cd /kaggle/
!git clone https://github.com/comfyanonymous/ComfyUI
%cd ComfyUI
!echo -= Updating ComfyUI =-
!git pull
!pip install -r requirements.txt
%cd /kaggle/ComfyUI/custom_nodes
![ -f "ComfyUI-Manager/check.sh" ] && chmod 755 ComfyUI-Manager/check.sh
![ -f "ComfyUI-Manager/scan.sh" ] && chmod 755 ComfyUI-Manager/scan.sh
![ -f "ComfyUI-Manager/node_db/dev/scan.sh" ] && chmod 755 ComfyUI-Manager/node_db/dev/scan.sh
![ -f "ComfyUI-Manager/scripts/install-comfyui-venv-linux.sh" ] && chmod 755 ComfyUI-Manager/scripts/install-comfyui-venv-linux.sh
![ -f "ComfyUI-Manager/scripts/install-comfyui-venv-win.bat" ] && chmod 755 ComfyUI-Manager/scripts/install-comfyui-venv-win.bat
![ ! -d ComfyUI-Manager ] && echo -= Initial setup ComfyUI-Manager =- && git clone https://github.com/ltdrdata/ComfyUI-Manager
%cd ComfyUI-Manager
!git pull
%cd /kaggle/ComfyUI
!pip install GitPython
!python custom_nodes/ComfyUI-Manager/cm-cli.py restore-dependencies
#!mkdir ./models/animatediff_models
#!wget -c "https://huggingface.co/wangfuyun/AnimateLCM/resolve/main/AnimateLCM_sd15_t2v.ckpt" -O ./models/animatediff_models/AnimateLCM_sd15_t2v.ckpt

In [ ]:
#models
#!wget -c "https://huggingface.co/wangfuyun/AnimateLCM/resolve/main/AnimateLCM_sd15_t2v_lora.safetensors" -O ./models/loras/AnimateLCM_sd15_t2v_lora.safetensors
#!wget -c "https://huggingface.co/Lykon/LoRA/resolve/7b44164cabdc9a4f34b4ef27f508dacde0a540e0/animemix_v3_offset.safetensors" -O ./models/loras/animemix_v3_offset.safetensors
#!wget -c "https://huggingface.co/naonovn/Lora/resolve/main/add_detail.safetensors" -O ./models/loras/add_detail.safetensors

#!wget -c "https://civitai.com/api/download/models/117037?type=Model&format=SafeTensor&size=full&fp=fp32&token=fde84741db38e41c3f38e6a10b7840a9" -O ./models/checkpoints/Juggernaut_finalbakedvae.safetensors
!wget -c "https://civitai.com/api/download/models/782002?type=Model&format=SafeTensor&size=full&fp=fp16&token=fde84741db38e41c3f38e6a10b7840a9" -O ./models/checkpoints/Juggernaut-XL_v9.safetensors

#text to video
!wget -c "https://huggingface.co/Kijai/WanVideo_comfy/resolve/main/Wan2_1-T2V-1_3B_bf16.safetensors" -O ./models/diffusion_models/Wan2_1-T2V_1_3B_bf16.safetensors
!wget -c "https://huggingface.co/Kijai/WanVideo_comfy/resolve/main/Wan2_1-T2V-1_3B_fp32.safetensors" -O ./models/diffusion_models/Wan2_1-T2V_1_3B_fp32.safetensors
#vace from kijai
!wget -c "https://huggingface.co/Kijai/WanVideo_comfy/resolve/main/Wan2_1_VACE_1_3B_preview_bf16.safetensors" -O ./models/diffusion_models/Wan2_1_VACE_1_3B_preview_bf16.safetensors
!wget -c "https://huggingface.co/Kijai/WanVideo_comfy/resolve/main/Wan2_1-VACE_module_14B_bf16.safetensors" -O ./models/diffusion_models/Wan2_1-VACE_module_14B_bf16.safetensors
#vace from wan
!wget -c "https://huggingface.co/ali-vilab/VACE-Wan2.1-1.3B-Preview/resolve/main/diffusion_pytorch_model.safetensors" -O ./models/diffusion_models/VACE-Wan2_1_1_3B_Preview.safetensors
# text encoder
!wget -c "https://huggingface.co/Kijai/WanVideo_comfy/resolve/main/umt5-xxl-enc-bf16.safetensors" -O ./models/text_encoders/umt5-xxl-enc-bf16.safetensors
#!wget -c "https://huggingface.co/Comfy-Org/Wan_2.1_ComfyUI_repackaged/resolve/main/split_files/text_encoders/umt5_xxl_fp8_e4m3fn_scaled.safetensors" -O ./models/text_encoders/umt5_xxl_fp8_e4m3fn_scaled.safetensors
#VAE
!wget -c "https://huggingface.co/Comfy-Org/Wan_2.1_ComfyUI_repackaged/resolve/main/split_files/text_encoders/umt5_xxl_fp16.safetensors" -O ./models/text_encoders/umt5_xxl_fp16.safetensors
!wget -c "https://huggingface.co/Kijai/WanVideo_comfy/resolve/main/Wan2_1_VAE_fp32.safetensors" -O ./models/vae/Wan2_1_VAE_fp32.safetensors
!wget -c "https://huggingface.co/Kijai/WanVideo_comfy/resolve/main/Wan2_1_VAE_bf16.safetensors" -O ./models/vae/Wan2_1_VAE_bf16.safetensors
#Asset
!gdown --id 1Pb9nswpIqeKMO0cKI1IHAkTwSvYSioS4

In [ ]:
!pip install av

In [ ]:
import subprocess
import threading
import time
import socket
import sys
from IPython.display import clear_output
import urllib.request
from tqdm.auto import tqdm
import re
import os

# Define the original ComfyUI output folder
COMFY_OUTPUT_DIR = "/kaggle/ComfyUI/output"

# Define a safe location in /kaggle/working for persisted outputs
PERSISTENT_OUTPUT_DIR = "/kaggle/working/comfyui_outputs"

# Create persistent output dir if not exists
os.makedirs(PERSISTENT_OUTPUT_DIR, exist_ok=True)

# Create a symlink if not already existing
if not os.path.islink(COMFY_OUTPUT_DIR):
    if os.path.exists(COMFY_OUTPUT_DIR):
        # Remove old dir if it exists
        subprocess.run(["rm", "-rf", COMFY_OUTPUT_DIR])
    # Create a symlink pointing COMFY_OUTPUT_DIR to PERSISTENT_OUTPUT_DIR
    os.symlink(PERSISTENT_OUTPUT_DIR, COMFY_OUTPUT_DIR)

# Global event to signal when we should exit
exit_event = threading.Event()

def iframe_thread(port):
    try:
        while not exit_event.is_set():
            time.sleep(0.5)
            sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
            result = sock.connect_ex(('127.0.0.1', port))
            if result == 0:
                break
            sock.close()
        
        if exit_event.is_set():
            return
            
        print("\nComfyUI finished loading, trying to launch localtunnel (if it gets stuck here localtunnel is having issues)\n")
        
        try:
            print("The password/endpoint ip for localtunnel is:", urllib.request.urlopen('https://ipv4.icanhazip.com', timeout=10).read().decode('utf8').strip("\n"))
        except Exception as e:
            print(f"Could not get external IP: {e}")
        
        try:
            lt_process = subprocess.Popen(["lt","--host http://loca.lt", "--port", f"{port}"], 
                                         stdout=subprocess.PIPE, 
                                         stderr=subprocess.PIPE, 
                                         text=True)
            while not exit_event.is_set():
                line = lt_process.stdout.readline()
                if not line:
                    break
                print(line, end='')
        except FileNotFoundError:
            print("Error: 'lt' command not found. Make sure localtunnel is installed.")
            print("Try installing it first with: !npm install -g localtunnel")
    except Exception as e:
        print(f"Error in iframe thread: {e}")

def run_main():
    script_path = "/kaggle/ComfyUI/main.py"
    if not os.path.exists(script_path):
        print(f"Error: Script not found at {script_path}")
        return None
    
    try:
        main_process = subprocess.Popen([sys.executable, script_path, "--dont-print-server"],
                                      stdout=subprocess.PIPE,
                                      stderr=subprocess.STDOUT,
                                      text=True,
                                      bufsize=1)
        
        def output_reader(process):
            try:
                pbar = None
                progress_pattern = re.compile(r'(\d+)%\|.*?\| (\d+)/(\d+)')
                
                while not exit_event.is_set():
                    line = process.stdout.readline()
                    if not line:
                        break
                    
                    match = progress_pattern.search(line)
                    if match:
                        percentage, current, total = map(int, match.groups())
                        if pbar is None:
                            pbar = tqdm(total=total, unit='it')
                        pbar.n = current
                        pbar.refresh()
                    else:
                        print(f"main.py: {line}", end='', flush=True)
                
                if pbar:
                    pbar.close()
            except Exception as e:
                print(f"Error in output reader: {e}")
                
        threading.Thread(target=output_reader, args=(main_process,), daemon=True).start()
        return main_process
    except Exception as e:
        print(f"Error starting main process: {e}")
        return None

def run_all_processes():
    print("Starting ComfyUI and localtunnel...")
    
    # Start the main.py process
    main_process = run_main()
    if not main_process:
        print("Failed to start main process. Please check error messages above.")
        return
    
    # Start the iframe thread
    iframe_thread_obj = threading.Thread(target=iframe_thread, args=(8188,))
    iframe_thread_obj.daemon = False  # Make this a non-daemon thread
    iframe_thread_obj.start()
    
    print("Processes started. Waiting for URL...")
    
    # This will keep the main process alive until keyboard interrupt
    try:
        while main_process.poll() is None:
            time.sleep(1)
    except KeyboardInterrupt:
        print("Interrupted by user")
    finally:
        exit_event.set()
        print("Shutting down...")
        # Try to terminate the main process if it's still running
        if main_process.poll() is None:
            main_process.terminate()
            main_process.wait(timeout=5)

# Run all processes with a blocking call that keeps the script running
if __name__ == "__main__":
    run_all_processes()
    print("Processes have completed. You can now run the next cell.")